In [22]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import StructuredTool, Tool, BaseTool
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import Type

In [3]:
cd ..

d:\LEARN\LangChain Basis


d:\LEARN\LangChain Basis\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [24]:
def greet_user(name: str) -> str:
    """Greets the user by name."""
    return f"Hello, {name}!"

greet_user_tool = Tool(
    name="GreetUser",
    func=greet_user,
    description="Greets the user by name."
)

In [25]:
# Pydantic models for tool arguments
# Define a Pydantic model to specify the input schema for tools that need more structured input.
class ReverseStringArgs(BaseModel):
    text: str = Field(description="Text to be reversed")

# Tool with One Parameter using args_schema (multiple parameters is okay too)
# Use the args_schema parameter to specify the input schema using a Pydantic model.
@tool(args_schema=ReverseStringArgs)
def reverse_string(text: str) -> str:
    """Reverses the given string."""
    return text[::-1]

In [26]:
def concatenate_strings(a: str, b: str) -> str:
    """Concatenates two strings."""
    return a + b

# Pydantic model for tool arguments
class ConcatenateStringsArgs(BaseModel):
    a: str = Field(description="First string")
    b: str = Field(description="Second string")

# Use StructuredTool for more complex functions that require multiple input parameters.
# StructuredTool allows us to define an input schema using Pydantic, ensuring proper validation and description.
concatenate_strings_tool = StructuredTool.from_function(
    name="ConcatenateStrings",  # Name of the tool
    func=concatenate_strings,  # Function to execute
    description="Concatenates two strings.",  # Description of the tool
    args_schema=ConcatenateStringsArgs,  # Schema defining the tool's input arguments
)

In [27]:
# Pydantic models for tool arguments
class SimpleSearchInput(BaseModel):
    query: str = Field(description="should be a search query")

# Custom tool with only custom input
class SimpleSearchTool(BaseTool):
    name = "simple_search"
    description = "useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel] = SimpleSearchInput

    def _run(
        self,
        query: str,
    ) -> str:
        """Use the tool."""
        from tavily import TavilyClient

        client = TavilyClient()
        results = client.search(query=query)
        return f"Search results for: {query}\n\n\n{results}\n"

In [28]:
tools = [greet_user_tool, reverse_string, concatenate_strings_tool, SimpleSearchTool()]

In [29]:
# Initialize a ChatOpenAI model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

# Pull the prompt template from the hub
prompt = hub.pull("hwchase17/openai-tools-agent")

# Create the ReAct agent using the create_tool_calling_agent function
agent = create_tool_calling_agent(
    llm=llm,  # Language model to use
    tools=tools,  # List of tools available to the agent
    prompt=prompt,  # Prompt template to guide the agent's responses
)

# Create the agent executor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,  # The agent to execute
    tools=tools,  # List of tools available to the agent
    verbose=True,  # Enable verbose logging
    handle_parsing_errors=True,  # Handle parsing errors gracefully
)

d:\LEARN\LangChain Basis\.venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [19]:
# Test the agent with sample queries
response = agent_executor.invoke({"input": "Greet Alice"})
print("Response for 'Greet Alice':", response)

response = agent_executor.invoke({"input": "Reverse the string 'hello'"})
print("Response for 'Reverse the string hello':", response)

response = agent_executor.invoke({"input": "Concatenate 'hello' and 'world'"})
print("Response for 'Concatenate hello and world':", response)



> Entering new AgentExecutor chain...

Invoking: `GreetUser` with `Alice`


Hello, Alice!Hello, Alice! 


> Finished chain.
Response for 'Greet Alice': {'input': 'Greet Alice', 'output': 'Hello, Alice! \n'}


> Entering new AgentExecutor chain...

Invoking: `reverse_string_tool` with `{'text': 'hello'}`


ollehThe reversed string is olleh. 


> Finished chain.
Response for 'Reverse the string hello': {'input': "Reverse the string 'hello'", 'output': 'The reversed string is olleh. \n'}


> Entering new AgentExecutor chain...

Invoking: `ConcatenateStrings` with `{'a': 'hello', 'b': 'world'}`


helloworldhelloworld 


> Finished chain.
Response for 'Concatenate hello and world': {'input': "Concatenate 'hello' and 'world'", 'output': 'helloworld \n'}


In [30]:
response = agent_executor.invoke({"input": "Search for Apple Intelligence"})
print("Response for 'Search for Apple Intelligence':", response)



> Entering new AgentExecutor chain...

Invoking: `simple_search` with `{'query': 'Apple Intelligence'}`


Search results for: Apple Intelligence


{'query': 'Apple Intelligence', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Apple Intelligence - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Apple_Intelligence', 'content': 'Apple Intelligence is an artificial intelligence platform developed by Apple Inc. for iOS, iPadOS, and macOS devices. It features writing tools, image generation, Siri overhaul, ChatGPT integration, and more.', 'score': 0.99978846, 'raw_content': None}, {'title': 'Introducing Apple Intelligence for iPhone, iPad, and Mac', 'url': 'https://www.apple.com/newsroom/2024/06/introducing-apple-intelligence-for-iphone-ipad-and-mac/', 'content': 'Apple Intelligence is a personal intelligence system that combines generative models with personal context to deliver useful and relevant intelligence. It powers new features for writing, comm